# AirControl, control PC with hands

In [1]:
from aircontrol.utils import get_window, numlock_off
from aircontrol.const import (
    BaseOptions,
    GestureRecognizer,
    GestureRecognizerOptions,
    VisionRunningMode,
    model_path,
)
from aircontrol.params import window_aware_gestures
from aircontrol.models.midas import MidasModel
import mediapipe as mp
import cv2 as cv
import numpy as np
import time
import os

## Callback

When a gesture is detected, if the opened window has been specified in the file ```params.py``` the program will execute the assigned action; if not, the called function will be one of the "General" dictionary.

The check on the x median point of the hand landmarks is used to determine the position on the x axis of the hand and, if defined, call the position-aware function.

In [2]:
def action_callback(result, output_image: mp.Image, timestamp_ms: int):
    global window_aware_gestures
    global key_pressed
    if not key_pressed:
        try:
            window, sub_window = get_window()
            sign = result.gestures[0][0].category_name
            if sign != "none" and sign != "":
                try:
                    func = window_aware_gestures[window][sub_window][sign]
                except KeyError:  # if action or window not specified in the dict
                    try:
                        func = window_aware_gestures[window][sign]
                    except KeyError:
                        func = window_aware_gestures["General"][sign]
                numlock_off()
                if callable(func):
                    func()

                else:
                    pos = result.hand_landmarks[0]
                    x_values = [d.x for d in pos]
                    xmid = np.median(x_values)
                    if xmid > 0.55:
                        func["Right"]()
                    elif xmid < 0.45:
                        func["Left"]()

                    else:
                        func["Center"]()
            key_pressed = True
        except:
            pass

## MediaPipe Configuration

In [3]:
options = GestureRecognizerOptions(
    base_options=BaseOptions(model_asset_path=os.path.join(os.getcwd(), model_path)),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=action_callback,
)

recognizer = GestureRecognizer.create_from_options(options)

## Improving recognition with MiDaS

For every frame:

1. A depth mask is obtained and scaled from 0 to 1 (where 0 means far from the cam objective).
2. The mask is multiplied by the original frame so that every far element becomes darker.

In [ ]:
midas = MidasModel()

## The detection loop

In [ ]:
cam = cv.VideoCapture(0)
print("Press 'q' to quit")
key_pressed = False
while True:
    success, frame = cam.read()
    frame = cv.flip(frame, 1)

    if not success:
        print("Camera Frame not available")
        continue

    adjusted_frame = midas.predict_adjust(frame)

    frame_mp = mp.Image(image_format=mp.ImageFormat.SRGB, data=adjusted_frame)

    timestamp_seconds = time.time()

    frame_timestamp_ms = int(timestamp_seconds * 1000)

    if not key_pressed:
        recognizer.recognize_async(frame_mp, frame_timestamp_ms)

    if key_pressed:
        time.sleep(2)
        key_pressed = False
    cv.imshow("Show Video", adjusted_frame)

    if cv.waitKey(20) & 0xFF == ord("q"):
        cam.release()
        break

cam.release()
cv.destroyAllWindows()